In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import time 

import matplotlib.pyplot as plt

from keras import Sequential
from keras.layers import Dense, Dropout, InputLayer, Lambda, Input, Flatten
from keras.models import Sequential
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.preprocessing.image import load_img, img_to_array

from sklearn.model_selection import train_test_split

读取数据集标签和测试集样例

In [ ]:
labels_di = pd.read_csv('/kaggle/input/dog-breed-identification/labels.csv')
samples = pd.read_csv('/kaggle/input/dog-breed-identification/sample_submission.csv')

print('Num of images in train dataset: {}'.format(len(labels_di)))
print('Num of images in test dataset: {}'.format(len(samples)))

规格化图像数据集，返回图像数据集与对应的标签

In [ ]:
def images_to_array(directory, label_dataframe, target_size = (331, 331, 3)):
    images = np.zeros([len(label_dataframe), target_size[0], target_size[1], target_size[2]], dtype=np.uint8)
    img = ''
    for ix, image_name in enumerate(label_dataframe['id'].values):
        img_dir = os.path.join(directory, image_name + '.jpg')
        img = load_img(img_dir, target_size = target_size)
        images[ix] = img_to_array(img)
    del img
    label_dict = dict(enumerate(label_dataframe['breed'].unique()))
    return images, label_dict

规格化训练集数据

In [ ]:
train_images, labels = images_to_array('/kaggle/input/dog-breed-identification/train', labels_di[:])

查看前八张图片

In [ ]:
plt.figure(figsize = (10, 10))
for ix, image in enumerate(train_images[:8]):
    plt.subplot(2, 4, ix + 1)
    plt.imshow(image / 255.0)
    plt.xticks([])
    plt.yticks([]) 

定义预测模型

In [ ]:
def get_feature(model_name, preprocess_input, images, target_size = (331,331,3)):
    base_model = model_name(input_shape = target_size, include_top=False, pooling = 'avg')

    model = Sequential()
    model.add(InputLayer(input_shape = target_size))
    model.add(Lambda(preprocess_input))
    model.add(base_model)

    feature = model.predict(images)
    
    print('feature-map shape: {}'.format(feature.shape))
    return feature

定义ResNetV2模型

In [ ]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input

resnet_preprocess = preprocess_input
resnet_feature = get_feature(InceptionResNetV2, resnet_preprocess, train_images)

定义NASNetLarge模型

In [ ]:
from keras.applications.nasnet import NASNetLarge, preprocess_input
nasnet_preprocessor = preprocess_input
nasnet_features = get_feature(NASNetLarge, nasnet_preprocessor, train_images)

联合两个模型来进行图像分类

In [ ]:
final_features = np.concatenate([nasnet_features, resnet_feature], axis = 1)
print('final features shape: {}'.format(final_features.shape))
del train_images, nasnet_features, resnet_feature

将标签转换为字典类型

In [ ]:
class_to_index = dict({labels[ix]:ix for ix in labels.keys()})
index_to_class = labels

In [ ]:
labels = labels_di['breed'].map(class_to_index)

定义模型

In [ ]:
def create_model(features_shape = 1024):
    model = Sequential()
    model.add(InputLayer(input_shape = (features_shape, )))
    #model.add(Dense(4096, activation = 'relu'))
    model.add(Dropout(0.8))
    model.add(Dense(len(class_to_index), activation = 'softmax'))
    
    model.compile(loss = 'sparse_categorical_crossentropy', optimizer ='Adam', metrics = ['accuracy'])
    return model


查看模型

In [ ]:
model = create_model(final_features.shape[1])
model.summary()

对模型进行训练

In [ ]:
model.fit(final_features, labels, epochs = 64, validation_split = 0.1)

将测试图片转换为固定形状

In [ ]:
def images_to_array(directory, label_dataframe, target_size = (331, 331,3)):
    images = np.zeros([len(label_dataframe), target_size[0], target_size[1], target_size[2]], dtype=np.uint8)
    img = ''
    for ix, image_name in enumerate(label_dataframe['id'].values):
        img_dir = os.path.join(directory, image_name + '.jpg')
        img = load_img(img_dir, target_size = target_size)
        images[ix] = img_to_array(img)
    del img
    return images

In [ ]:
test_images = images_to_array('/kaggle/input/dog-breed-identification/test', samples)

In [ ]:
resnet_feature = get_feature(InceptionResNetV2, resnet_preprocess, test_images)
nasnet_features = get_feature(NASNetLarge, nasnet_preprocessor, test_images)

In [ ]:
final_features = np.concatenate([nasnet_features, resnet_feature], axis = 1)
print('final features shape: {}'.format(final_features.shape))

进行预测

In [ ]:
prediction = model.predict(final_features)
submission = pd.DataFrame({'id':samples.id})
prediction = pd.DataFrame(prediction)
prediction.columns = class_to_index.keys()

将结果进行保存

In [ ]:
submission = pd.concat([submission, prediction], axis = 1)
submission.to_csv('submission.csv', index = False)